### Pydantic 

The most important library for Data Validations & Parsing 

Allows `strict typing` by using type annotations.

In [2]:
import pydantic 

print('pydantic version:', pydantic.__version__)

pydantic version: 2.5.3


1. Data Validation without Pydantic

In [4]:
class User: 
    def __init__(self, id: int, name='Jane Doe'):
        
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an integer, but got {type(id).__name__}')
        
        if not isinstance(name, str):
            raise TypeError(f'Expected name to be a string, but got {type(name).__name__}')
        
        self.id = id 
        self.name = name 

Expected id to be an integer, but got str


In [6]:
# case - 1: id is an integer 
try: 
    user = User(id=123)
    print("Data is valid!")
except TypeError as e: 
    print(e)

Data is valid!


In [7]:
# case - 2: id is an string 
try: 
    user = User(id='123')
    print("Data is valid!") 
except TypeError as e:
    print(e)

Expected id to be an integer, but got str


#### 2. Data validation with Pydantic 

In [13]:
from pydantic import BaseModel

class User(BaseModel):
    id: int 
    name: str = 'Jane Doe' 

In [14]:
# case - 1: id is an integer 
user = User(id=123) 

In [18]:
# case - 2: id is an string 
user = User(id='dada') 

ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='dada', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing

In [20]:
# case - 3: id is an integer but in quotes 
# ex: '123'
user = User(id='123')  

# it will automatically converts it into integer if it is an real integer but in str format!
# It will parse string as an integer (Pydantic feature)

In [16]:
user.id 

1243

In [17]:
type(user.id)

int

In [21]:
print(user.model_fields)

{'id': FieldInfo(annotation=int, required=True), 'name': FieldInfo(annotation=str, required=False, default='Jane Doe')}


In [23]:
print(user.model_fields_set) 
# it won't print default values in model field set

{'id'}


In [25]:
# 2nd object 
user_2 = User(id=123, name='karthik')

user_2

User(id=123, name='karthik')

In [27]:
user_2.model_dump()

{'id': 123, 'name': 'karthik'}

In [28]:
# model fields 
user_2.model_fields

{'id': FieldInfo(annotation=int, required=True),
 'name': FieldInfo(annotation=str, required=False, default='Jane Doe')}

In [29]:
user_2.model_fields_set

{'id', 'name'}

Converting Data models into data objects

In [30]:
# dict object 
user_2.model_dump()

{'id': 123, 'name': 'karthik'}

In [31]:
# json object 
user_2.model_dump_json()

'{"id":123,"name":"karthik"}'

In [33]:
# json schema 
user_2.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'name': {'default': 'Jane Doe', 'title': 'Name', 'type': 'string'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}

#### Nested Models 

In [34]:
from typing import List, Optional 
from pydantic import BaseModel 

class Food(BaseModel):
    name: str 
    price: float 
    ingredients: Optional[List[str]] = None 

class Restaurant(BaseModel):
    name: str 
    location: str 
    foods: List[Food] 

In [35]:
restaurant_instances = Restaurant(name='Tasty Bites', 
                                  location="123, Fifth Avenue",
                                  foods=[
                                      {'name': 'Cheese Pizza', 'price': 12.50, 'ingredients':['Cheese', 'Tomato Sauce', 'Flour']},
                                      {'name':'Veggie Burger', 'price': 8.99}
                                  ])


In [36]:
print(restaurant_instances)

name='Tasty Bites' location='123, Fifth Avenue' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Flour']), Food(name='Veggie Burger', price=8.99, ingredients=None)]


In [39]:
obj = restaurant_instances.model_dump()

In [40]:
obj 

{'name': 'Tasty Bites',
 'location': '123, Fifth Avenue',
 'foods': [{'name': 'Cheese Pizza',
   'price': 12.5,
   'ingredients': ['Cheese', 'Tomato Sauce', 'Flour']},
  {'name': 'Veggie Burger', 'price': 8.99, 'ingredients': None}]}

In [41]:
type(obj)

dict

### Pydantic for Email Validation

for this we need to install pydantic[email] sub-module from pydantic.

because it won't come with pydantic module 

In [42]:
!pip install pydantic[email]

   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/307.7 kB 1.9 MB/s eta 0:00:01
   ------------------- -------------------- 153.6/307.7 kB 1.5 MB/s eta 0:00:01
   ------------------------------- -------- 245.8/307.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  307.2/307.7 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 1.6 MB/s eta 0:00:00


In [43]:
from typing import List 
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

In [48]:
class Address(BaseModel): 
    street: str 
    city: str
    state: str 
    zip_code: str 

class Employee(BaseModel):
    name: str 
    position: str 
    email: EmailStr

class Owner(BaseModel):
    name: str 
    email: EmailStr

class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9- ']+$")
    owner: Owner
    address: Address
    employee: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery: bool 
    website: HttpUrl 

In [53]:
# creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty bites",
    owner={
        'name':'John Doe',
        'email': 'john.deo@gmail.com'
    },
    address={
        'street': '123, Floor 1, Suite',
        'city': 'San Francisco',
        'state': 'CA',
        'zip_code': '12345'
    },
    employee=[
        {
            'name': 'Mike Json',
            'position': 'Chef',
            'email': 'mike.json@example.com'
        },
        {
            'name':'Shyam sundar',
            'position': 'waiter',
            'email': 'shyam.sunder@example.com'
        }  
    ],
    number_of_seats=40, 
    delivery=True, 
    website="http://tastybites.com"
)


print("Data is in valid format!!")

Data is in valid format!!


In [54]:
restaurant_instance

Restaurant(name='Tasty bites', owner=Owner(name='John Doe', email='john.deo@gmail.com'), address=Address(street='123, Floor 1, Suite', city='San Francisco', state='CA', zip_code='12345'), employee=[Employee(name='Mike Json', position='Chef', email='mike.json@example.com'), Employee(name='Shyam sundar', position='waiter', email='shyam.sunder@example.com')], number_of_seats=40, delivery=True, website=Url('http://tastybites.com/'))

In [55]:
restaurant_instance.model_dump()

{'name': 'Tasty bites',
 'owner': {'name': 'John Doe', 'email': 'john.deo@gmail.com'},
 'address': {'street': '123, Floor 1, Suite',
  'city': 'San Francisco',
  'state': 'CA',
  'zip_code': '12345'},
 'employee': [{'name': 'Mike Json',
   'position': 'Chef',
   'email': 'mike.json@example.com'},
  {'name': 'Shyam sundar',
   'position': 'waiter',
   'email': 'shyam.sunder@example.com'}],
 'number_of_seats': 40,
 'delivery': True,
 'website': Url('http://tastybites.com/')}

### Field Validator

In [57]:
from pydantic import BaseModel, field_validator, EmailStr

In [64]:
# Our Data schema 
class Owner(BaseModel):
    name: str 
    email: EmailStr

    @field_validator('name')
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v: 
            raise ValueError("Owner name must contain a space!") 
        return v.title() 
    
# we can return what ever we want:  v.upper() 

In [65]:
try: 
    owner_instance = Owner(name='john dope', email='johndope@email.com')
    print(owner_instance)
except ValueError as e:
    print(e)

name='John Dope' email='johndope@email.com'


In [70]:
a = 10

In [67]:
type(a)

int

In [68]:
isinstance(a, int)

True

In [69]:
isinstance(a, str)

False

### Model Validator 

https://www.youtube.com/watch?v=7aBRk_JP-qY